## 📘 HDB Resale Flat Prices

### 📌 Notebook Description

- **Team:** Team A  
- **Members:** Ben, Shazlin, Alan  
- **Project Name:** HDB Resale Flat Data Engineering Pipeline
- **Description:** Implements automated data ingestion from data.gov.sg and performs dataset merging to produce a unified, analysis-ready dataset.
- **Data Artifacts:**  
    - `/DataLake/<raw files>`  
    - `/Staging/Main.csv`

### 📦 Import Required Libraries

In [1]:
import pandas as pd

#---Customized-----------------------------------------
import control_output
pd.set_option("display.float_format", "{:,.2f}".format)
control_output.css

### 🔧 Data Processing

In [2]:
#hdb_data = pd.read_csv('datasets/Main.csv', low_memory=False, usecols=lambda col: not col.startswith("Unnamed"))
hdb_data = pd.read_csv('../Project-HDB-Store/staging/Main.csv', low_memory=False, usecols=lambda col: not col.startswith("Unnamed"))
hdb_data = hdb_data.rename(columns={'month': 'year_month'})
hdb_data["year_month"] = pd.to_datetime(hdb_data["year_month"], format="%Y-%m-%d")
hdb_data.set_index('year_month', inplace=True)

# Important: sort by index
hdb_data = hdb_data.sort_index()

hdb_data_final = hdb_data['2000-01':'2022-12-31']

df_stat_montly = pd.read_csv('../Project-HDB-Store/staging/stat_monthly.csv',
    index_col="year_month",      # use year_month as index during load
    parse_dates=["year_month"]   # convert to datetime automatically
)

df_stat_yearly = pd.read_csv('../Project-HDB-Store/staging/stat_yearly.csv',
    index_col="year",      # use year_month as index during load
    parse_dates=["year"]   # convert to datetime automatically
)

df_merged = hdb_data_final.join(df_stat_montly, how="left")
df_merged = df_merged.join(df_stat_yearly, how="left")
#df_merged.to_csv("staging/Main_final.csv")
df_merged.to_csv("../Project-HDB-Store/staging/Main_final.csv", index=True, index_label="year_month")

### 📋 Sample Data (head)

In [3]:
df_merged.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,lease_commence_date,storey_range,block,remaining_lease,...,nearest_mrt,nearest_distance_to_mrt,remaining_years,price_per_sqm,birth,marriages,divorces,unemployment,inflation,gdp
2000-01-01,ANG MO KIO,3RM,IMPROVED,69.00,ANG MO KIO AVE 4,"147,000.00",1986,07 TO 09,170,85,...,mayflower,0.28,85,"2,130.43",3585,1602,396,3.60,1.34,"96,076,539,925.74"
2000-01-01,BISHAN,4RM,MODEL A,113.00,BISHAN ST 22,"382,500.00",1992,07 TO 09,249,91,...,ang mo kio,1.16,91,"3,384.96",3585,1602,396,3.60,1.34,"96,076,539,925.74"
2000-01-01,BEDOK,3RM,IMPROVED,68.00,NEW UPP CHANGI RD,"166,000.00",1985,01 TO 03,57,84,...,tanah merah,0.62,84,"2,441.18",3585,1602,396,3.60,1.34,"96,076,539,925.74"
2000-01-01,PASIR RIS,4RM,MODEL A,110.00,PASIR RIS ST 53,"338,000.00",1995,16 TO 18,574,94,...,pasir ris,0.38,94,"3,072.73",3585,1602,396,3.60,1.34,"96,076,539,925.74"
2000-01-01,PASIR RIS,4RM,MODEL A,108.00,PASIR RIS ST 53,"307,800.00",1995,04 TO 06,575,94,...,pasir ris,0.33,94,"2,850.00",3585,1602,396,3.60,1.34,"96,076,539,925.74"


### 📋 Check Retrieve Dates

In [41]:
sorted_dates = df_merged.index.unique()
datexs = list((sorted_dates[:12].values)) + list((sorted_dates[-12:].values))

for d in datexs:
    print(d)

2000-01-01T00:00:00.000000000
2000-02-01T00:00:00.000000000
2000-03-01T00:00:00.000000000
2000-04-01T00:00:00.000000000
2000-05-01T00:00:00.000000000
2000-06-01T00:00:00.000000000
2000-07-01T00:00:00.000000000
2000-08-01T00:00:00.000000000
2000-09-01T00:00:00.000000000
2000-10-01T00:00:00.000000000
2000-11-01T00:00:00.000000000
2000-12-01T00:00:00.000000000
2022-01-01T00:00:00.000000000
2022-02-01T00:00:00.000000000
2022-03-01T00:00:00.000000000
2022-04-01T00:00:00.000000000
2022-05-01T00:00:00.000000000
2022-06-01T00:00:00.000000000
2022-07-01T00:00:00.000000000
2022-08-01T00:00:00.000000000
2022-09-01T00:00:00.000000000
2022-10-01T00:00:00.000000000
2022-11-01T00:00:00.000000000
2022-12-01T00:00:00.000000000


### 📤 Output

In [6]:
print(f"Total HDB Resale Transation: {len(df_merged)}")

Total HDB Resale Transation: 599650
